## Brian Sohn, bs3358

In [1]:
import numpy as np
import pandas as pd
import pymc as pm
import arviz as az

In [26]:
data = pd.read_csv('data.csv', index_col = [0])
Y_0 = data[data['group']==0]['no_purchase']
Y_1 = data[data['group']==1]['no_purchase']
data

,group,no_purchase
0,1,0
1,1,3
2,1,0
3,1,0
4,1,0
...,...,...
9995,0,6
9996,0,3
9997,0,7
9998,0,0


In [69]:
model = pm.Model()

with model:
    ## prob of someone being a whale is centered around 0.5%
    p_w_0 = pm.Normal('p_w_0', mu=pm.math.logit(0.005), tau=1)
    p_w = pm.Deterministic('p_w', pm.math.invlogit(p_w_0))
    
    ## group 0   
    # prob of someone being zero-buyer given that someone is not a whale
    pzero_0_0 = pm.Normal('pzero_0_0', mu=pm.math.logit(0.85), tau=1)
    pzero_0 = pm.Deterministic('pzero_0', pm.math.invlogit(pzero_0_0))
    # mean purchase for casual buyers
    mu_0 = pm.HalfNormal('mu_0', tau=1)
    # mean difference of purchase whale - casual
    mu_w_diff = pm.HalfNormal('mu_w_diff', tau=.01)
    # mixture model
    dist_0_w = pm.Poisson.dist(mu=mu_0+mu_w_diff) # whales
    dist_0_n = pm.ZeroInflatedPoisson.dist(psi=1-pzero_0, mu=mu_0) # non-whales
    obs_0 = pm.Mixture('obs_0', w=[p_w, 1-p_w], comp_dists=[dist_0_w, dist_0_n], observed=Y_0)
    
    ## group 1
    # prob of someone being zero-buyer given that someone is not a whale
    pzero_1_0 = pm.Normal('pzero_1_0', mu=pm.math.logit(0.85), tau=1)
    pzero_1 = pm.Deterministic('pzero_1', pm.math.invlogit(pzero_1_0))
    # mean purchase for casual buyers
    mu_1 = pm.HalfNormal('mu_1', tau=1)
    # since whales are price inelastic, use mu_w above for group 1 too
    # mixture model
    dist_1_w = pm.Poisson.dist(mu=mu_0+mu_w_diff) # whales
    dist_1_n = pm.ZeroInflatedPoisson.dist(psi=1-pzero_1, mu=mu_1) # non-whales
    obs_1 = pm.Mixture('obs_1', w=[p_w, 1-p_w], comp_dists=[dist_1_w, dist_1_n], observed=Y_1)
    
    
    idata = pm.sample()
    
f = az.summary(idata)
f

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [p_w_0, pzero_0_0, mu_0, mu_w_diff, pzero_1_0, mu_1]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 209 seconds.


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
p_w_0,-6.016,0.202,-6.400,-5.639,0.003,0.002,6417.0,3325.0,1.0
pzero_0_0,1.700,0.040,1.625,1.776,0.001,0.000,5792.0,3178.0,1.0
pzero_1_0,1.773,0.042,1.691,1.848,0.001,0.000,5572.0,3196.0,1.0
mu_0,4.305,0.076,4.163,4.443,0.001,0.001,5114.0,3296.0,1.0
mu_w_diff,59.088,1.611,56.139,62.172,0.021,0.015,5830.0,3105.0,1.0
mu_1,3.044,0.071,2.911,3.174,0.001,0.001,5551.0,3062.0,1.0
p_w,0.002,0.000,0.002,0.003,0.000,0.000,6417.0,3325.0,1.0
pzero_0,0.845,0.005,0.835,0.855,0.000,0.000,5792.0,3178.0,1.0
pzero_1,0.855,0.005,0.845,0.864,0.000,0.000,5572.0,3196.0,1.0


In [77]:
p_w = np.array(idata.posterior.p_w).flatten()
pzero_0 = np.array(idata.posterior.pzero_0).flatten()
pzero_1 = np.array(idata.posterior.pzero_1).flatten()
mu_0 = np.array(idata.posterior.mu_0).flatten()
mu_1 = np.array(idata.posterior.mu_1).flatten()
mu_w_diff = np.array(idata.posterior.mu_w_diff).flatten()

In [78]:
rev_0 = (p_w * (mu_0 + mu_w_diff) + (1 - p_w) * (1 - pzero_0) * mu_0) * 3.99
rev_1 = (p_w * (mu_0 + mu_w_diff) + (1 - p_w) * (1 - pzero_1) * mu_1) * 4.99

In [87]:
loss_0 = np.where(rev_0 >= rev_1, 0, rev_1 - rev_0).mean()
loss_1 = np.where(rev_0 <= rev_1, 0, rev_0 - rev_1).mean()

print(f"Expected Loss per person when price is $3.99: ${loss_0:.4f}")
print(f"Expected Loss per person when price is $4.99: ${loss_1:.4f}")

Expected Loss per person when price is $3.99: $0.0011
Expected Loss per person when price is $4.99: $0.2896


In [ ]:
# cf) tried to calculate individual probabilities for p_w and pzero, but ran into multiple problems

In [20]:
# model = pm.Model()
# Y_0 = data[data['group']==0]['no_purchase']
# Y_1 = data[data['group']==1]['no_purchase']


# with model:
    
#     ## prob of someone being a whale is centered around 0.5%
#     p_w_0 = pm.Normal('p_w_0', mu=pm.math.logit(0.005), tau=1)
#     p_w = pm.Deterministic('p_w', pm.math.invlogit(p_w_0))
    
#     ## group 0   
#     # prob of someone being zero-buyer given that someone is not a whale
#     pzero_0_0 = pm.Normal('pzero_0_0', mu=pm.math.logit(0.85), tau=1)
#     pzero_0 = pm.Deterministic('pzero_0', pm.math.invlogit(pzero_0_0))
#     # mean purchase for casual buyers
#     mu_0 = pm.HalfNormal('mu_0', tau=10)
#     # mean difference of purchase whale - casual
#     mu_w_diff = pm.HalfNormal('mu_w_diff', tau=.01)
#     # mixture model
#     dist_0_w = pm.Poisson.dist(mu=mu_0+mu_w_diff) # whales
#     #dist_0_n = pm.ZeroInflatedPoisson.dist(psi=1-pzero_0, mu=mu_0) # non-whales
#     dist_0_c = pm.Poisson.dist(mu=mu_0) # casual buyers
#     dist_0_n = pm.Constant.dist(c=0) # non-buyers
#     #obs_0 = pm.Mixture('obs_0', w=[p_w, 1-p_w], comp_dists=[dist_0_w, dist_0_n], observed=data[data['group']==0]['no_purchase'])
#     whale_0_p = p_w * pm.math.exp(pm.logp(dist_0_w, Y_0))
#     casual_0_p = (1-p_w) * (1-pzero_0) * pm.math.exp(pm.logp(dist_0_c, Y_0))
#     nobuy_0_p = (1-p_w) * pzero_0 * pm.math.exp(pm.logp(dist_0_n, Y_0))
#     log0p = pm.math.log(whale_0_p + casual_0_p + nobuy_0_p)
#     p0_whale_i = pm.Deterministic('p0_whale_i', whale_0_p/pm.math.exp(log0p))
#     p0_casual_i = pm.Deterministic('p0_casual_i', casual_0_p/pm.math.exp(log0p))
#     obs_0 = pm.Potential('obs_0', pm.math.log(whale_0_p + casual_0_p + nobuy_0_p))
    
    
#     ## group 1
#     # prob of someone being zero-buyer given that someone is not a whale
#     pzero_1_0 = pm.Normal('pzero_1_0', mu=pm.math.logit(0.85), tau=1)
#     pzero_1 = pm.Deterministic('pzero_1', pm.math.invlogit(pzero_1_0))
#     # mean purchase for casual buyers
#     mu_1 = pm.HalfNormal('mu_1', tau=10)
#     # since whales are price inelastic, use mu_w above for group 1 too
#     # mixture model
#     dist_1_w = pm.Poisson.dist(mu=mu_0+mu_w_diff) # whales
#     #dist_1_n = pm.ZeroInflatedPoisson.dist(psi=1-pzero_1, mu=mu_1) # non-whales
#     dist_1_c = pm.Poisson.dist(mu=mu_1) # casual buyers
#     dist_1_n = pm.Constant.dist(c=0) # non-buyers
#     #obs_1 = pm.Mixture('obs_1', w=[p_w, 1-p_w], comp_dists=[dist_1_w, dist_1_n], observed=data[data['group']==1]['no_purchase'])
#     whale_1_p = p_w * pm.math.exp(pm.logp(dist_1_w, Y_1))
#     casual_1_p = (1-p_w) * (1-pzero_1) * pm.math.exp(pm.logp(dist_1_c, Y_1))
#     nobuy_1_p = (1-p_w) * pzero_1 * pm.math.exp(pm.logp(dist_1_n, Y_1))
#     log1p = pm.math.log(whale_1_p + casual_1_p + nobuy_1_p)
#     p1_whale_i = pm.Deterministic('p1_whale_i', whale_1_p/pm.math.exp(log1p))
#     p1_casual_i = pm.Deterministic('p1_casual_i', casual_1_p/pm.math.exp(log1p))
#     obs_1 = pm.Potential('obs_1', pm.math.log(whale_1_p + casual_1_p + nobuy_1_p))
    
#     idata = pm.sample()
    
# f = az.summary(idata)
# f

C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\pymc\distributions\discrete.py:1431: FutureWarning: pm.Constant has been deprecated. Use pm.DiracDelta instead.
  warnings.warn(
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [p_w_0, pzero_0_0, mu_0, mu_w_diff, pzero_1_0, mu_1]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 144 seconds.
C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\arviz\stats\diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\arviz\stats\diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\arviz\stats\diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\arviz\stats\diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_var

C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\arviz\stats\diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\arviz\stats\diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\arviz\stats\diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\arviz\stats\diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\arviz\s

C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\arviz\stats\diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\arviz\stats\diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\arviz\stats\diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\arviz\stats\diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\arviz\s

C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\arviz\stats\diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\arviz\stats\diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\arviz\stats\diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\arviz\stats\diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\arviz\s

...

C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\arviz\stats\diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\arviz\stats\diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\arviz\stats\diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\arviz\stats\diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\arviz\s

C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\arviz\stats\diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\arviz\stats\diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\arviz\stats\diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\arviz\stats\diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\arviz\s

C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\arviz\stats\diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\arviz\stats\diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\arviz\stats\diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\arviz\stats\diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\arviz\s

C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\arviz\stats\diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\arviz\stats\diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\arviz\stats\diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\arviz\stats\diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
C:\Users\sohnh\anaconda3\envs\pymc_env\lib\site-packages\arviz\s

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
p_w_0,-6.018,0.195,-6.376,-5.645,0.003,0.002,5067.0,3310.0,1.0
pzero_0_0,1.697,0.039,1.622,1.767,0.001,0.000,5552.0,3221.0,1.0
pzero_1_0,1.765,0.041,1.689,1.843,0.001,0.000,5086.0,3553.0,1.0
mu_0,4.098,0.072,3.968,4.233,0.001,0.001,5583.0,3202.0,1.0
mu_w_diff,59.269,1.661,56.240,62.383,0.021,0.015,6032.0,3607.0,1.0
...,...,...,...,...,...,...,...,...,...
p1_casual_i[4995],1.000,0.000,1.000,1.000,0.000,0.000,4000.0,4000.0,NaN
p1_casual_i[4996],0.009,0.001,0.008,0.011,0.000,0.000,5012.0,3096.0,1.0
p1_casual_i[4997],0.009,0.001,0.008,0.011,0.000,0.000,5012.0,3096.0,1.0
p1_casual_i[4998],0.009,0.001,0.008,0.011,0.000,0.000,5012.0,3096.0,1.0
